In [1]:
%matplotlib inline

import pandas as pd 
import numpy as np

In [2]:
df_trips = pd.read_csv("data/generated-marketplace-trips.csv")
df_passengers = pd.read_csv("data/generated-marketplace-passengers.csv")
df_pilots = pd.read_csv("data/generated-marketplace-pilots.csv")
df_planets = pd.read_csv("data/generated-marketplace-planets.csv")

## misc

In [3]:
planet_names = np.append(df_planets["name"].values, "All")
planet_options = [{"label": p, "value": p.lower()} for p in planet_names]

user_types = ["Pilot", "Passenger"]
user_options = [{"label": t, "value": t.lower()} for t in user_types]

df_trips["trip_requested_dt"] = pd.to_datetime(df_trips["trip_requested"], unit="s")
df_trips["trip_requested_month"] = df_trips["trip_requested_dt"].dt.month
trips_months = np.sort(df_trips["trip_requested_month"].unique())
month_options = {str(m): str(m) for m in trips_months}
min_month = trips_months.min()
max_month = trips_months.max()

df_trips_week = df_trips["trip_requested_dt"].dt.week
df_trips_week = df_trips_week.value_counts().sort_index()

df_trips["trip_duration"] = df_trips["trip_ended"] - df_trips["trip_started"]

In [4]:
def id_to_planet(ids):
    return df_planets.iloc[ids]["name"].values

## get_month_summary

In [5]:
def get_month_summary(df, month):
    trips, pilots, passengers = None, None, None
    if month in list(df["trip_requested_month"]): # compare with list because `0 in series` returns True
        dff = df[df["trip_requested_month"] == month]
        trips = len(dff)
        pilots = len(dff["pilot"].unique())
        passengers = len(dff["passenger"].unique())
    return np.array([trips, pilots, passengers])

### show tests

In [6]:
print(get_month_summary(df_trips, -1))
print(get_month_summary(df_trips, 0))
print(get_month_summary(df_trips, 1))
print(get_month_summary(df_trips, 2))
print(get_month_summary(df_trips, 12))
print(get_month_summary(df_trips, 13))

[None None None]
[None None None]
[1 1 1]
[8 7 7]
[4200  100  966]
[None None None]


## generate_summary_table

In [7]:
def generate_summary_table(df, month):
    cm = get_month_summary(df_trips, month)
    pm = get_month_summary(df_trips, month - 1)
    df = pd.DataFrame({
        "Current": cm, "Previous": pm if pm[0] else "-", 
        "M/M Change": np.round((cm * 1.0 / pm), 2) if pm[0] else "-"
    }, index=["Trips", "Pilots", "Passengers"],
    columns=["Current", "Previous", "M/M Change"])
    return df

### show tests

In [8]:
generate_summary_table(df_trips, 0)

,Current,Previous,M/M Change
Trips,None,-,-
Pilots,None,-,-
Passengers,None,-,-


In [9]:
generate_summary_table(df_trips, 3)

,Current,Previous,M/M Change
Trips,53,8,6.62
Pilots,19,7,2.71
Passengers,44,7,6.29


In [10]:
generate_summary_table(df_trips, 12)

,Current,Previous,M/M Change
Trips,4200,1968,2.13
Pilots,100,92,1.09
Passengers,966,746,1.29


## filter_df

In [11]:
def filter_df(df, month="all", planet="all"):
    dff = df.copy(deep=True)
    dff.loc[:, "planet"] = id_to_planet(dff["planet"])
    dff = dff[dff["trip_requested_month"] == month] if month != "all" else dff
    dff = dff[dff["planet"].str.lower() == planet] if planet != "all" else dff
    return dff

### show tests

In [12]:
filter_df(df_trips[["planet", "trip_requested_month"]]).head(3)

,planet,trip_requested_month
0,Coruscant,12
1,Duro,10
2,Hosnian Prime,12


In [13]:
filter_df(df_trips[["planet", "trip_requested_month"]], planet="coruscant", month=1).head(3)

,planet,trip_requested_month
1020,Coruscant,1


## get_table_pilots

In [14]:
filtered_columns = [
    "pilot", "planet", "trip_completed", "trip_duration",
    "price", "pilot_rating"
]

In [15]:
def pad_df(df, columns, size, spacer="-"):
    blanks = [spacer for i in range(size)]
    dff = pd.DataFrame({i: blanks for i in columns})
    for idx, row in df[:min(size, len(df))].iterrows():
        dff.iloc[idx, :] = row
    return dff

In [16]:
def get_table_pilots(df, month="all", planet="all", top=3):
    """Returns an html.Table object containing pilot statistics,
    filtered by month and planet."""

    filtered_columns = [
        "pilot", "planet", "trip_completed", "trip_duration",
        "price", "pilot_rating"
    ]

    dff = df.copy(deep=True)
    dff = filter_df(dff, month=month, planet=planet)
    dff = dff[filtered_columns]
    dff_ranked = dff.groupby("pilot").agg({
        "planet": "max",
        "trip_completed": "sum",
        "pilot_rating": "mean",
        "price": "sum",
        "trip_duration": "mean"
    }).sort_values("trip_completed", ascending=False)
    dff_ranked = dff_ranked.reset_index()
    dff_ranked["price"] = dff_ranked["price"].map(
        lambda x: "{:,.2f}".format(x))
    dff_ranked["trip_duration"] = dff_ranked["trip_duration"].map(
        lambda x: "{}:{:02d}".format(*divmod(int(x), 60)))
    dff_ranked["pilot_rating"] = dff_ranked["pilot_rating"].round(2)
    dff_ranked = pad_df(dff_ranked, filtered_columns, top)
    dff_ranked = dff_ranked[filtered_columns]
    
    return dff_ranked[:top]

### show tests

In [17]:
get_table_pilots(df_trips, "all", "coruscant")

,pilot,planet,trip_completed,trip_duration,price,pilot_rating
0,76,Coruscant,135,32:43,"121,431.00",4.38
1,6,Coruscant,134,34:10,"133,065.00",4.28
2,29,Coruscant,134,32:06,"124,122.00",4.37


In [18]:
get_table_pilots(df_trips, 12, "all")

,pilot,planet,trip_completed,trip_duration,price,pilot_rating
0,56,Coruscant,105,32:36,"100,980.00",4.31
1,15,Hosnian Prime,101,31:39,"94,404.00",4.49
2,1,Corellia,92,30:24,"82,144.00",4.33


In [19]:
get_table_pilots(df_trips, 1, "all")

,pilot,planet,trip_completed,trip_duration,price,pilot_rating
0,6,Coruscant,1,55:23,"1,994.00",1
1,-,-,-,-,-,-
2,-,-,-,-,-,-


## get_table_users

In [20]:
def get_table_users(df, user_type, month="all", planet="all", top=3):
    """Returns an html.Table object containing user statistics,
    filtered by month and planet."""
    
    user_rating = user_type + "_rating"  # example: "pilot_rating"
    user_label = user_type.capitalize() + " ID"  # example: "Pilot ID"

    filtered_columns = [
        user_type, "planet", "trip_completed", "trip_duration",
        "price", user_rating
    ]

    dff = df.copy(deep=True)
    dff = filter_df(dff, month=month, planet=planet)
    dff = dff[filtered_columns]
    dff_ranked = dff.groupby(user_type).agg({
        "planet": "max",
        "trip_completed": "sum",
        user_rating: "mean",
        "price": "sum",
        "trip_duration": "mean"
    }).sort_values("trip_completed", ascending=False)
    dff_ranked = dff_ranked.reset_index()
    dff_ranked["price"] = dff_ranked["price"].map(
        lambda x: "{:,.2f}".format(x))
    dff_ranked["trip_duration"] = dff_ranked["trip_duration"].map(
        lambda x: "{}:{:02d}".format(*divmod(int(x), 60)))
    dff_ranked[user_rating] = dff_ranked[user_rating].round(2)
    dff_ranked = pad_df(dff_ranked, filtered_columns, top)
    dff_ranked = dff_ranked[filtered_columns]
    
    return dff_ranked[:top]

### show_tests

In [21]:
get_table_users(df_trips, "pilot", month=12)

,pilot,planet,trip_completed,trip_duration,price,pilot_rating
0,56,Coruscant,105,32:36,"100,980.00",4.31
1,15,Hosnian Prime,101,31:39,"94,404.00",4.49
2,1,Corellia,92,30:24,"82,144.00",4.33


In [22]:
get_table_users(df_trips, "passenger", month=12)

,passenger,planet,trip_completed,trip_duration,price,passenger_rating
0,888,Coruscant,17,27:33,"13,151.00",4.47
1,588,Coruscant,17,28:52,"13,772.00",3.94
2,277,Coruscant,16,34:16,"15,789.00",4.47


## get_table_users with average

In [23]:
def get_table_users(df, user_type, month="all", planet="all", top=3):
    """Returns an html.Table object containing user statistics,
    filtered by month and planet."""
    
    user_rating = user_type + "_rating"  # example: "pilot_rating"
    user_label = user_type.capitalize()  # example: "Pilot"

    filtered_columns = [
        user_type, "planet", "trip_completed", "trip_duration",
        "price", user_rating
    ]

    dff = df.copy(deep=True)
    dff = filter_df(dff, month=month, planet=planet)
    dff = dff[filtered_columns]
    dff_ranked = dff.groupby(user_type).agg({
        "planet": "max",
        "trip_completed": "sum",
        user_rating: "mean",
        "price": "sum",
        "trip_duration": "mean"
    }).sort_values("trip_completed", ascending=False)
    dff_ranked = dff_ranked.reset_index()
    dff_ranked.loc["Average " + user_label] = dff_ranked.mean()
    
    dff_ranked["trip_completed"] = dff_ranked["trip_completed"].round(0)
    dff_ranked["price"] = dff_ranked["price"].map(
        lambda x: "{:,.2f}".format(x))
    dff_ranked["trip_duration"] = dff_ranked["trip_duration"].map(
        lambda x: "{}:{:02d}".format(*divmod(int(x), 60)))
    dff_ranked[user_rating] = dff_ranked[user_rating].round(2)
    
    dff_ranked_average = dff_ranked.loc["Average " + user_label]
    dff_ranked_average = pd.DataFrame(dff_ranked_average).T
    dff_ranked_average.loc["Average " + user_label, user_type] = "-"
    dff_ranked_average.loc["Average " + user_label, "planet"] = "-"
    
    dff_ranked = pad_df(dff_ranked, filtered_columns, top)
    dff_ranked = pd.concat((dff_ranked, dff_ranked_average))
    dff_ranked = dff_ranked[filtered_columns]
    
    return dff_ranked

In [24]:
get_table_users(df_trips, "pilot", month=12)

,pilot,planet,trip_completed,trip_duration,price,pilot_rating
0,56,Coruscant,105,32:36,"100,980.00",4.31
1,15,Hosnian Prime,101,31:39,"94,404.00",4.49
2,1,Corellia,92,30:24,"82,144.00",4.33
Average Pilot,-,-,40,32:44,"37,533.34",4.39


In [25]:
get_table_users(df_trips, "passenger", month=12)

,passenger,planet,trip_completed,trip_duration,price,passenger_rating
0,888,Coruscant,17,27:33,"13,151.00",4.47
1,588,Coruscant,17,28:52,"13,772.00",3.94
2,277,Coruscant,16,34:16,"15,789.00",4.47
Average Passenger,-,-,4,32:37,"3,885.44",4.26
